# FLUX.1-dev with Diffuse: Modular Sampling Framework

This notebook demonstrates how to use FLUX.1-dev {cite}`flux2024` with the **Diffuse** sampling framework - showcasing the power of researching sampling algorithms WITHOUT dealing with training complexity.

## The Research Opportunity

Most exciting diffusion research isn't about training models - it's about the **algorithms built on top**:

- 🎨 **Image editing** (InstructPix2Pix, DiffEdit, Imagic)
- 🖼️ **Inpainting & outpainting** (RePaint, Blended Diffusion)
- 🔍 **Inverse problems** (DPS, RED-diff, medical imaging)
- 🎯 **Controllable generation** (ControlNet-style, regional control)
- 📐 **Novel sampling methods** (better integrators, adaptive schedules)
- ⚡ **Distillation & acceleration** (consistency models, few-step sampling)

All of these need **modular sampling infrastructure**. But most researchers rebuild it from scratch every time.

**Diffuse solves this**: Load pre-trained models → Experiment with sampling → Focus on YOUR research.

## What We'll Explore

1. **Flow Matching Models** - Understanding FLUX as a velocity field predictor
2. **Modular Components** - Swap timers, integrators, guidance without touching model code
3. **Quality Comparisons** - Visual side-by-side across configurations
4. **Stochastic vs Deterministic** - Adding controlled randomness for diversity
5. **Research Velocity** - Test ideas in hours, not weeks

## What is FLUX.1-dev?

**FLUX.1-dev** {cite}`flux2024` is a state-of-the-art text-to-image model trained using **flow matching** {cite}`Liu2022` {cite}`Lipman2022` (also called rectified flow). Unlike traditional diffusion models that learn to denoise images, FLUX learns a **velocity field** that transforms noise into images along straight paths.

### Flow Matching Background

As detailed in the [Diffusion Crash Course](diffusion_crash_course.rst), flow matching uses the straight-line interpolation path:

$$
x_t = (1-t)x_0 + t\varepsilon, \quad \varepsilon\sim\mathcal{N}(0,I), \quad t \in [0, 1]
$$

where $x_0$ is clean data and $\varepsilon$ is Gaussian noise. FLUX {cite}`flux2024` learns a **velocity field** $v_\theta(x_t, t, c)$ conditioned on text embeddings $c$ that defines the flow ODE (see Eq. :eq:`eq:flow_ode` in the crash course):

$$
\frac{dx}{dt} = v_\theta(x_t, t, c)
$$

### Why This Matters for Research

This ODE formulation means you can:
- ✅ **Swap integrators** (Euler, Heun, DPM++, DDIM) without retraining
- ✅ **Change discretization schedules** (uniform, adaptive, learned)
- ✅ **Add stochasticity** (churning, noise injection) for diversity
- ✅ **Apply to inverse problems** (inpainting, super-resolution, deblurring)
- ✅ **Compose guidance methods** (classifier-free, DPS, custom)

**No model training needed.** Just load weights and experiment with sampling.

**Diffuse** makes this modular and easy. That's the point.

## Setup

In [ ]:
from pathlib import Pathimport numpy as npimport matplotlib.pyplot as pltfrom matplotlib.gridspec import GridSpecimport jaximport jax.numpy as jnpfrom PIL import Image# FLUX components (standalone - no triax dependency!)from diffuse.examples.flux_dev.run_flux_inference import (    FluxModelLoader,    FluxConditionedNetwork,    _latent_shapes,)from diffuse.examples.flux_dev.utils import FluxTimer# Diffuse componentsfrom diffuse.denoisers.denoiser import Denoiserfrom diffuse.diffusion.sde import Flowfrom diffuse.integrator.deterministic import (    DDIMIntegrator,    EulerIntegrator,    HeunIntegrator,    DPMpp2sIntegrator,)from diffuse.integrator.stochastic import EulerMaruyamaIntegratorfrom diffuse.predictor import Predictorfrom diffuse.timer.base import VpTimerprint(f"JAX devices: {jax.devices()}")

## Configuration

Set paths and generation parameters. Update `CHECKPOINT_DIR` to point to your FLUX model checkpoint.

In [ ]:
# Paths - UPDATE THIS TO MATCH YOUR SETUP
CHECKPOINT_DIR = Path("/path/to/flux/checkpoint")  # Contains transformer/vae/clip_text/t5_text

# Generation parameters
PROMPT = "A serene landscape with mountains at sunset, highly detailed, photorealistic"
HEIGHT = 512
WIDTH = 512
NUM_STEPS = 20
GUIDANCE_SCALE = 4.0
SEED = 42

print(f"Prompt: {PROMPT}")
print(f"Resolution: {WIDTH}x{HEIGHT}")
print(f"Steps: {NUM_STEPS}")

## Load FLUX Model

This section demonstrates Diffuse's **separation of concerns** - one of the key design principles that enables rapid experimentation.

### The Three Stages

1. **Model Loading** - `FluxModelLoader` handles checkpoints, tokenizers, text encoders  
2. **Text Conditioning** - `prepare_conditioned_network` encodes prompt, returns velocity field  
3. **Sampling** - Modular components (Timer, Integrator, Denoiser) handle generation

### What does `prepare_conditioned_network` do?

FLUX was trained using flow matching to predict **velocity fields**. This function:

- **Tokenizes your prompt** using CLIP and T5 tokenizers  
- **Encodes text** through CLIP (pooled embeddings) and T5 (sequence embeddings)
- **Loads the FLUX transformer** onto GPU
- **Returns a conditioned velocity field** `v(x_t, t)` - a function mapping (latents + time) → velocities

Think of it as "baking" your text prompt into the velocity field. Now you can sample from this field using **any integrator** without touching text encoders again.

### Why This Design Matters for Research

- ✅ **Text encoders offloaded** after encoding (saves memory)
- ✅ **Clean velocity field interface** for sampling experiments
- ✅ **Swap sampling components** without re-encoding  
- ✅ **Zero loading complexity** for your research code

This is the "simple pipeline, no loading craziness" philosophy in action. You focus on sampling algorithms, not infrastructure.

In [ ]:
# Load model
loader = FluxModelLoader(checkpoint_dir=CHECKPOINT_DIR, verbose=True)

# Prepare conditioned velocity field
conditioned = loader.prepare_conditioned_network(
    prompt=PROMPT,
    negative_prompt=None,
    guidance_scale=GUIDANCE_SCALE,
    height=HEIGHT,
    width=WIDTH,
)

print(f"\nConditioned network ready (dtype={conditioned.dtype})")

## Helper Functions: Modular Sampling with Diffuse

The key to Diffuse's power is **separation of concerns**. Each component has **one job**:

- **Timer**: Decides WHEN to evaluate the model (discretization schedule)
- **Predictor**: Wraps the velocity field `v(x_t, t)` from FLUX
- **Integrator**: Decides HOW to step from `x_t` to `x_{t-1}` (numerical ODE solver)
- **Denoiser**: Orchestrates the full sampling loop

This modularity is the entire point: **change one component, everything else stays the same**.

### The No-Training Research Paradigm

Want to test a new integrator idea? Just swap one line.  
Want to try a different schedule? Just swap one line.  
Want to add guidance? Just swap the Denoiser.

No model retraining. No loading spaghetti. **Just your research idea**.

In [ ]:
def run_generation(
    conditioned_network: FluxConditionedNetwork,
    timer,
    integrator_class,
    height: int,
    width: int,
    num_steps: int,
    seed: int,
) -> jax.Array:
    """Generate image using modular Diffuse components."""
    _, transformer_hw = _latent_shapes(height, width)
    image_seq_len = transformer_hw[0] * transformer_hw[1]

    # Set dynamic shift for FluxTimer if needed
    if isinstance(timer, FluxTimer) and timer.use_dynamic_shift:
        timer.set_image_seq_len(image_seq_len)

    # Modular component assembly
    flow = Flow(tf=1.0)
    predictor = Predictor(
        model=flow,
        network=conditioned_network.network_fn,
        prediction_type="velocity",
    )
    integrator = integrator_class(model=flow, timer=timer)
    denoiser = Denoiser(
        integrator=integrator,
        model=flow,
        predictor=predictor,
        x0_shape=(transformer_hw[0], transformer_hw[1], conditioned_network.in_channels),
    )

    # Run sampling
    key = jax.random.PRNGKey(seed)
    state, _ = denoiser.generate(
        rng_key=key,
        n_steps=num_steps,
        n_particles=1,
        keep_history=False,
    )
    return state.integrator_state.position.astype(conditioned_network.dtype)


def decode_and_display(latents: jax.Array, loader: FluxModelLoader) -> np.ndarray:
    """Decode latents to RGB image."""
    images = loader.decode_latents(latents)
    return images[0]


def plot_comparison(images: dict, title: str, figsize=(15, 10)):
    """Plot grid of images for comparison."""
    n = len(images)
    cols = min(3, n)
    rows = (n + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()

    for idx, (name, img) in enumerate(images.items()):
        axes[idx].imshow(img)
        axes[idx].set_title(name, fontsize=12, fontweight="bold")
        axes[idx].axis("off")

    for idx in range(len(images), len(axes)):
        axes[idx].axis("off")

    plt.suptitle(title, fontsize=16, fontweight="bold", y=0.98)
    plt.tight_layout()
    plt.show()

## Part 1: Timer Comparison

Timers control the **time discretization** $t \in [0, 1]$. Different schedules allocate more steps to different noise levels.

### VpTimer
Linear discretization: $t_i = t_f + \frac{i}{N}(\epsilon - t_f)$

### FluxTimer
Applies a Möbius transformation to bias sampling toward low-noise regions:

$$
\sigma_{\text{shifted}}(t) = \frac{\mu \cdot \sigma(t)}{1 + (\mu - 1) \cdot \sigma(t)}
$$

- **Static mode**: Fixed $\mu = 1.15$ (FLUX default)
- **Dynamic mode**: Resolution-adaptive $\mu(L)$ based on sequence length $L$

The Möbius shift allocates more steps to fine details (low noise), improving quality.

In [ ]:
# Create timers
vp_timer = VpTimer(n_steps=NUM_STEPS, eps=1e-3, tf=1.0)
flux_timer_static = FluxTimer(
    n_steps=NUM_STEPS, eps=1e-3, tf=1.0, shift=1.15, use_dynamic_shift=False
)
flux_timer_dynamic = FluxTimer(
    n_steps=NUM_STEPS, eps=1e-3, tf=1.0, shift=1.15, use_dynamic_shift=True
)

# Visualize schedules
_, transformer_hw = _latent_shapes(HEIGHT, WIDTH)
image_seq_len = transformer_hw[0] * transformer_hw[1]
flux_timer_dynamic.set_image_seq_len(image_seq_len)

steps = np.arange(NUM_STEPS + 1)
vp_schedule = [vp_timer(s) for s in steps]
flux_static_schedule = [flux_timer_static(s) for s in steps]
flux_dynamic_schedule = [flux_timer_dynamic(s) for s in steps]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(steps, vp_schedule, "o-", label="VpTimer", linewidth=2)
plt.plot(steps, flux_static_schedule, "s-", label=r"FluxTimer (static, $\mu=1.15$)", linewidth=2)
plt.plot(steps, flux_dynamic_schedule, "^-", label=f"FluxTimer (dynamic, $\\mu={flux_timer_dynamic._mu:.3f}$)", linewidth=2)
plt.xlabel("Step $i$", fontsize=12)
plt.ylabel("Time $t_i$", fontsize=12)
plt.title("Timer Schedules", fontsize=14, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
diff_static = np.diff(flux_static_schedule)
diff_dynamic = np.diff(flux_dynamic_schedule)
diff_vp = np.diff(vp_schedule)
plt.plot(steps[:-1], -diff_vp, "o-", label="VpTimer", linewidth=2)
plt.plot(steps[:-1], -diff_static, "s-", label="FluxTimer (static)", linewidth=2)
plt.plot(steps[:-1], -diff_dynamic, "^-", label="FluxTimer (dynamic)", linewidth=2)
plt.xlabel("Step $i$", fontsize=12)
plt.ylabel("Step Size $-\\Delta t_i$", fontsize=12)
plt.title("Step Size Distribution", fontsize=14, fontweight="bold")
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nResolution: {WIDTH}x{HEIGHT} → {image_seq_len} tokens")
print(f"Dynamic shift μ: {flux_timer_dynamic._mu:.3f}")

### Generate with Different Timers

We use `DDIMIntegrator` {cite}`Song2020b` (see crash course section on DDIM) to isolate the effect of the timer.

In [ ]:
print("Generating with VpTimer...")
latents_vp = run_generation(conditioned, vp_timer, DDIMIntegrator, HEIGHT, WIDTH, NUM_STEPS, SEED)
img_vp = decode_and_display(latents_vp, loader)

print("Generating with FluxTimer (static)...")
latents_flux_static = run_generation(
    conditioned, flux_timer_static, DDIMIntegrator, HEIGHT, WIDTH, NUM_STEPS, SEED
)
img_flux_static = decode_and_display(latents_flux_static, loader)

print("Generating with FluxTimer (dynamic)...")
latents_flux_dynamic = run_generation(
    conditioned, flux_timer_dynamic, DDIMIntegrator, HEIGHT, WIDTH, NUM_STEPS, SEED
)
img_flux_dynamic = decode_and_display(latents_flux_dynamic, loader)

timer_images = {
    "VpTimer": img_vp,
    "FluxTimer (Static)": img_flux_static,
    "FluxTimer (Dynamic)": img_flux_dynamic,
}

plot_comparison(
    timer_images,
    f"Timer Comparison (DDIM, {NUM_STEPS} steps)",
    figsize=(18, 6),
)

**Expected Differences:**
- **VpTimer**: Uniform steps, may miss fine details
- **FluxTimer (Static)**: More low-noise steps, better details
- **FluxTimer (Dynamic)**: Resolution-adaptive, optimal for multi-resolution workflows

## Part 2: Integrator Comparison

### What Are Integrators?

FLUX gives us a velocity field `v(x_t, t)` - it tells us "which direction to move" at each point. An **integrator** is the numerical method we use to follow that velocity field from noise (t=1) to image (t=0).

Think of it like GPS navigation:
- **Velocity field** = GPS telling you "turn left, go straight, turn right"  
- **Integrator** = how you follow those directions (fast but imprecise vs slow but accurate)

### Why Integrators Matter for Quality

Different integrators can significantly affect **visual quality**. For example:
- DPM++2S often produces **sharper reflections** (like sunlight on water)
- Heun preserves **fine texture details** better than Euler
- Euler-Maruyama adds **controlled stochasticity** for diverse samples

This is pure sampling research - no model retraining, just algorithmic improvements.

### Integrators We'll Compare

**DDIM** {cite}`Song2020b` (see crash course section on DDIM)
- Fast, deterministic, well-tested
- Good default choice for most applications

**Euler** (First-Order)
- Simplest: just follow the velocity directly
- Fast but can accumulate errors over many steps  
- Good for quick previews

**Heun** (Second-Order)
- "Look ahead" method: predicts next step, corrects itself
- 2x slower (two model evaluations per step) but more accurate
- Better detail preservation

**DPM++2S** {cite}`Lu2022` (Second-Order Optimized)
- Like Heun but with optimized stability
- Works in log-space for better numerical precision
- Often produces best quality - **notice water reflections, fine textures**

**Euler-Maruyama** (Stochastic)
- Adds controlled noise during sampling
- Generates diverse samples even with same seed
- Great for exploration and multiple variations

### Key Insight: Quality vs Speed Trade-off

- **Same steps**: Heun ≈ DPM++2S > DDIM > Euler (quality)
- **Same compute**: DDIM at 40 steps ≈ Heun at 20 steps
- **For FLUX**: DPM++2S often produces noticeably better fine details

Let's see the visual differences using FluxTimer (the default FLUX schedule):

In [ ]:
# Use FluxTimer for fair comparison
timer_for_comparison = FluxTimer(
    n_steps=NUM_STEPS, eps=1e-3, tf=1.0, shift=1.15, use_dynamic_shift=False
)

integrators = [
    ("DDIM", DDIMIntegrator),
    ("Euler", EulerIntegrator),
    ("Heun", HeunIntegrator),
    ("DPM++2S", DPMpp2sIntegrator),
    ("Euler-Maruyama", EulerMaruyamaIntegrator),
]

integrator_images = {}

for name, integrator_class in integrators:
    print(f"Generating with {name}...")
    latents = run_generation(
        conditioned, timer_for_comparison, integrator_class, HEIGHT, WIDTH, NUM_STEPS, SEED
    )
    img = decode_and_display(latents, loader)
    integrator_images[name] = img

plot_comparison(
    integrator_images,
    f"Integrator Comparison (FluxTimer, {NUM_STEPS} steps)",
    figsize=(20, 10),
)

### What to Look For
- **DDIM**: Solid baseline, clean images
- **Euler**: Slightly softer details
- **Heun**: Sharper than Euler
- **DPM++2S**: Often best quality (reflections, textures)
- **Euler-Maruyama**: Stochastic diversity

### Recommendations
- **Fast previews**: Euler or DDIM
- **Balanced**: DDIM
- **Max quality**: DPM++2S or Heun
- **Diversity**: Euler-Maruyama

## Part 3: Inverse Problems with DPS

So far we've done **unconditional generation** - going from noise to images guided only by text prompts. But one of the most powerful research areas is solving **inverse problems**: recovering clean images from degraded observations.

### What are Inverse Problems?

An inverse problem has this form:

$$
y = A(x) + n
$$

where:
- $x$ is the clean image we want to recover
- $A$ is a measurement operator (downsampling, masking, blurring, etc.)
- $y$ is our observed/degraded image
- $n$ is measurement noise

**Common examples:**
- **Inpainting**: $A$ masks out pixels → recover missing regions
- **Super-resolution**: $A$ downsamples image → recover high-res details
- **Deblurring**: $A$ applies blur kernel → recover sharp image
- **Compressed sensing**: $A$ is random projection → recover from few measurements
- **Medical imaging**: $A$ is CT/MRI physics → reconstruct from k-space

### How DPS Works {cite}`Chung2022`

DPS (Diffusion Posterior Sampling) elegantly solves inverse problems by:

1. **Taking a normal diffusion step** using the integrator (as if unconditional)
2. **Checking measurement consistency**: "Does our current estimate match the observed data $y$?"
3. **Applying a gradient correction** to push toward measurement-consistent solutions

Mathematically: $x_{t-1} = \text{step}(x_t, t) - \zeta \nabla_x \|y - A(x_t)\|^2$

The beauty: **you can use any integrator + any measurement operator without retraining**!

### Why This Matters for Research

This is THE example of "algorithms on top of models" research. With Diffuse:
- ✅ Implement your measurement operator (20 lines of code)
- ✅ Plug into DPS with any integrator
- ✅ Test immediately on FLUX (or any model)
- ✅ Compare against baselines instantly

No training. No model modification. Just your inverse problem algorithm.

Let's see it in action with two examples: **inpainting** and **super-resolution**.

In [ ]:
# Import DPS denoiser
from diffuse.denoisers.cond import DPSDenoiser
from diffuse.base_forward_model import MeasurementState, ForwardModel
from dataclasses import dataclass
from jax.numpy import Array

# Define forward models for FLUX images

@dataclass
class InpaintingMask(ForwardModel):
    """Inpainting forward model - masks out regions of the image."""
    mask_type: str = "rectangle"  # "rectangle" or "random"
    mask_ratio: float = 0.5  # Fraction of image to mask
    std: float = 0.01

    def create_mask(self, img_shape: tuple, key: jax.random.PRNGKey = None) -> Array:
        """Create a binary mask (1 = keep, 0 = remove)."""
        H, W, C = img_shape

        if self.mask_type == "rectangle":
            # Create rectangular mask in center
            mask_h = int(H * self.mask_ratio)
            mask_w = int(W * self.mask_ratio)
            start_h = (H - mask_h) // 2
            start_w = (W - mask_w) // 2

            mask = jnp.ones((H, W, C))
            mask = mask.at[start_h:start_h+mask_h, start_w:start_w+mask_w, :].set(0.0)

        elif self.mask_type == "random":
            # Random pixel mask
            if key is None:
                key = jax.random.PRNGKey(42)
            mask = jax.random.bernoulli(key, 1 - self.mask_ratio, shape=img_shape).astype(jnp.float32)

        return mask

    def apply(self, img: Array, measurement_state: MeasurementState) -> Array:
        """Apply mask to image."""
        mask = measurement_state.mask_history
        return img * mask

    def restore(self, img: Array, measurement_state: MeasurementState) -> Array:
        """Return the masked region (for gradient computation)."""
        mask = measurement_state.mask_history
        inv_mask = 1 - mask
        return img * inv_mask


@dataclass
class SuperResolution(ForwardModel):
    """Super-resolution forward model - downsamples then upsamples image."""
    scale_factor: int = 4  # Downsampling factor (4x4 = 16x fewer pixels)
    std: float = 0.01

    def apply(self, img: Array, measurement_state: MeasurementState) -> Array:
        """Downsample image using average pooling."""
        H, W, C = img.shape
        s = self.scale_factor

        # Reshape and average pool
        img_down = img.reshape(H//s, s, W//s, s, C).mean(axis=(1, 3))

        # Upsample back to original size using nearest neighbor
        img_up = jnp.repeat(jnp.repeat(img_down, s, axis=0), s, axis=1)

        return img_up

    def restore(self, img: Array, measurement_state: MeasurementState) -> Array:
        """Identity for super-res (no special adjoint needed for DPS)."""
        return img


print("✓ Forward models defined")

### Example 1: Inpainting

First, generate a reference image, then mask it and recover the missing regions using DPS.

In [ ]:
# Generate a reference image for inpainting
print("Generating reference image...")
INPAINT_PROMPT = "A cute cat sitting on a windowsill, photorealistic, detailed fur"
INPAINT_STEPS = 25

# Update conditioned network with new prompt
conditioned_inpaint = loader.prepare_conditioned_network(
    prompt=INPAINT_PROMPT,
    negative_prompt=None,
    guidance_scale=GUIDANCE_SCALE,
    height=HEIGHT,
    width=WIDTH,
)

# Generate reference image using best quality settings (DPM++2S + FluxTimer)
timer_inpaint = FluxTimer(n_steps=INPAINT_STEPS, eps=1e-3, tf=1.0, shift=1.15, use_dynamic_shift=False)
latents_ref = run_generation(
    conditioned_inpaint, timer_inpaint, DPMpp2sIntegrator, HEIGHT, WIDTH, INPAINT_STEPS, SEED
)
img_ref = decode_and_display(latents_ref, loader)

# Create inpainting mask and degraded observation
inpaint_model = InpaintingMask(mask_type="rectangle", mask_ratio=0.5, std=0.01)
mask_inpaint = inpaint_model.create_mask(img_ref.shape)
img_masked = img_ref * mask_inpaint

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img_ref)
axes[0].set_title("Original Image", fontsize=12, fontweight="bold")
axes[0].axis("off")

axes[1].imshow(mask_inpaint)
axes[1].set_title("Mask (white=keep, black=remove)", fontsize=12, fontweight="bold")
axes[1].axis("off")

axes[2].imshow(img_masked)
axes[2].set_title("Masked Observation (50% missing)", fontsize=12, fontweight="bold")
axes[2].axis("off")

plt.tight_layout()
plt.show()

print(f"Masked region: {(1 - mask_inpaint.mean()):.1%} of pixels")

In [ ]:
# Solve inpainting with DPS
print("Running inpainting with DPS...")

# Create measurement state (in latent space)
latents_masked = loader.encode_images(img_masked[None, ...])[0]  # Encode to latent space
mask_latent = jax.image.resize(
    mask_inpaint,
    shape=(latents_masked.shape[0], latents_masked.shape[1], mask_inpaint.shape[2]),
    method="nearest"
)

measurement_inpaint = MeasurementState(
    y=latents_masked,
    mask_history=mask_latent
)

# Create DPS denoiser with Heun integrator (good balance of quality and speed)
flow_inpaint = Flow(tf=1.0)
predictor_inpaint = Predictor(
    model=flow_inpaint,
    network=conditioned_inpaint.network_fn,
    prediction_type="velocity"
)

# Define inpainting forward model in latent space
@dataclass
class LatentInpainting(ForwardModel):
    std: float = 0.01

    def apply(self, x: Array, measurement_state: MeasurementState) -> Array:
        mask = measurement_state.mask_history
        return x * mask

    def restore(self, x: Array, measurement_state: MeasurementState) -> Array:
        mask = measurement_state.mask_history
        return x * (1 - mask)

latent_inpaint_model = LatentInpainting(std=0.01)

integrator_inpaint = HeunIntegrator(model=flow_inpaint, timer=timer_inpaint)
dps_inpaint = DPSDenoiser(
    integrator=integrator_inpaint,
    model=flow_inpaint,
    predictor=predictor_inpaint,
    forward_model=latent_inpaint_model,
    x0_shape=latents_masked.shape,
    zeta=0.3,  # Guidance strength
)

# Generate inpainted samples
key_inpaint = jax.random.PRNGKey(SEED + 100)
state_inpaint, _ = dps_inpaint.generate(
    key_inpaint,
    measurement_inpaint,
    n_steps=INPAINT_STEPS,
    n_particles=4,  # Generate 4 diverse solutions
    keep_history=False,
)

# Decode latents to images
latents_inpainted = state_inpaint.integrator_state.position
imgs_inpainted = loader.decode_latents(latents_inpainted.astype(conditioned_inpaint.dtype))

print(f"✓ Generated {len(imgs_inpainted)} inpainted results")

In [ ]:
# Visualize inpainting results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle("Inpainting with DPS: Multiple Solutions", fontsize=16, fontweight="bold")

# Row 1: Original, Masked, First result
axes[0, 0].imshow(img_ref)
axes[0, 0].set_title("Original", fontsize=12, fontweight="bold")
axes[0, 0].axis("off")

axes[0, 1].imshow(img_masked)
axes[0, 1].set_title("Masked (50% missing)", fontsize=12, fontweight="bold")
axes[0, 1].axis("off")

axes[0, 2].imshow(imgs_inpainted[0])
axes[0, 2].set_title("DPS Reconstruction #1", fontsize=12, fontweight="bold")
axes[0, 2].axis("off")

# Row 2: Additional diverse solutions
for i in range(3):
    if i < len(imgs_inpainted) - 1:
        axes[1, i].imshow(imgs_inpainted[i + 1])
        axes[1, i].set_title(f"DPS Reconstruction #{i + 2}", fontsize=12, fontweight="bold")
    axes[1, i].axis("off")

plt.tight_layout()
plt.show()

print("\\nNotice how DPS produces plausible completions that:")
print("- Preserve visible regions exactly")
print("- Fill in missing content coherently")
print("- Generate diverse solutions (stochastic sampling)")

### Example 2: Super-Resolution

Now let's tackle super-resolution: recovering high-resolution details from a low-resolution image.

In [ ]:
# Create super-resolution degradation
print("Creating super-resolution problem...")

# Use the same reference image or generate a new one
SR_PROMPT = "A mountain landscape at golden hour, highly detailed, sharp, photorealistic"
conditioned_sr = loader.prepare_conditioned_network(
    prompt=SR_PROMPT,
    negative_prompt=None,
    guidance_scale=GUIDANCE_SCALE,
    height=HEIGHT,
    width=WIDTH,
)

# Generate high-res reference
latents_hr = run_generation(
    conditioned_sr, timer_inpaint, DPMpp2sIntegrator, HEIGHT, WIDTH, INPAINT_STEPS, SEED + 1
)
img_hr = decode_and_display(latents_hr, loader)

# Create low-resolution observation (4x downsampling)
sr_model = SuperResolution(scale_factor=4, std=0.01)
img_lr_upsampled = sr_model.apply(img_hr, MeasurementState(y=None, mask_history=None))

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img_hr)
axes[0].set_title("High-Resolution Original", fontsize=12, fontweight="bold")
axes[0].axis("off")

axes[1].imshow(img_lr_upsampled)
axes[1].set_title("4x Downsampled (16x fewer pixels)", fontsize=12, fontweight="bold")
axes[1].axis("off")

# Create detail comparison boxes
h, w = img_hr.shape[:2]
crop = (h//4, h//4, 3*h//4, 3*w//4)  # Center crop
axes[2].imshow(img_hr[crop[0]:crop[2], crop[1]:crop[3]])
axes[2].set_title("Detail (original)", fontsize=12, fontweight="bold")
axes[2].axis("off")

plt.tight_layout()
plt.show()

print(f"Resolution: {HEIGHT}x{WIDTH} → {HEIGHT//4}x{WIDTH//4} (4x downsampling)")

In [ ]:
# Solve super-resolution with DPS
print("Running super-resolution with DPS...")

# Work in latent space
latents_lr = loader.encode_images(img_lr_upsampled[None, ...])[0]

measurement_sr = MeasurementState(
    y=latents_lr,
    mask_history=jnp.ones_like(latents_lr)  # Full observation (but downsampled)
)

# Define super-resolution forward model in latent space
@dataclass
class LatentSuperRes(ForwardModel):
    scale_factor: int = 4
    std: float = 0.01

    def apply(self, x: Array, measurement_state: MeasurementState) -> Array:
        """Downsample and upsample in latent space."""
        H, W, C = x.shape
        s = self.scale_factor

        # Average pool down
        x_down = x.reshape(H//s, s, W//s, s, C).mean(axis=(1, 3))
        # Upsample back
        x_up = jnp.repeat(jnp.repeat(x_down, s, axis=0), s, axis=1)
        return x_up

    def restore(self, x: Array, measurement_state: MeasurementState) -> Array:
        return x

latent_sr_model = LatentSuperRes(scale_factor=4, std=0.01)

# Create DPS denoiser
flow_sr = Flow(tf=1.0)
predictor_sr = Predictor(
    model=flow_sr,
    network=conditioned_sr.network_fn,
    prediction_type="velocity"
)

integrator_sr = HeunIntegrator(model=flow_sr, timer=timer_inpaint)
dps_sr = DPSDenoiser(
    integrator=integrator_sr,
    model=flow_sr,
    predictor=predictor_sr,
    forward_model=latent_sr_model,
    x0_shape=latents_lr.shape,
    zeta=0.5,  # Stronger guidance for super-res
)

# Generate super-resolved images
key_sr = jax.random.PRNGKey(SEED + 200)
state_sr, _ = dps_sr.generate(
    key_sr,
    measurement_sr,
    n_steps=INPAINT_STEPS,
    n_particles=3,  # Generate 3 variations
    keep_history=False,
)

# Decode
latents_sr = state_sr.integrator_state.position
imgs_sr = loader.decode_latents(latents_sr.astype(conditioned_sr.dtype))

print(f"✓ Generated {len(imgs_sr)} super-resolved results")

In [ ]:
# Visualize super-resolution results with detail crops
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle("Super-Resolution with DPS: 4x Upscaling", fontsize=16, fontweight="bold")

# Define crop region for detail view
crop = (HEIGHT//4, WIDTH//4, 3*HEIGHT//4, 3*WIDTH//4)

# Row 1: Full images
axes[0, 0].imshow(img_hr)
axes[0, 0].set_title("Original High-Res", fontsize=12, fontweight="bold")
axes[0, 0].axis("off")

axes[0, 1].imshow(img_lr_upsampled)
axes[0, 1].set_title("Low-Res (4x downsampled)", fontsize=12, fontweight="bold")
axes[0, 1].axis("off")

axes[0, 2].imshow(imgs_sr[0])
axes[0, 2].set_title("DPS Super-Resolved", fontsize=12, fontweight="bold")
axes[0, 2].axis("off")

# Row 2: Detail crops
axes[1, 0].imshow(img_hr[crop[0]:crop[2], crop[1]:crop[3]])
axes[1, 0].set_title("Original (detail)", fontsize=11)
axes[1, 0].axis("off")

axes[1, 1].imshow(img_lr_upsampled[crop[0]:crop[2], crop[1]:crop[3]])
axes[1, 1].set_title("Low-Res (detail) - blurry", fontsize=11)
axes[1, 1].axis("off")

axes[1, 2].imshow(imgs_sr[0][crop[0]:crop[2], crop[1]:crop[3]])
axes[1, 2].set_title("DPS (detail) - sharp!", fontsize=11)
axes[1, 2].axis("off")

# Row 3: Additional variations
for i in range(3):
    if i < len(imgs_sr):
        axes[2, i].imshow(imgs_sr[i])
        axes[2, i].set_title(f"DPS Variation #{i+1}", fontsize=11)
    axes[2, i].axis("off")

plt.tight_layout()
plt.show()

print("\\nNotice how DPS super-resolution:")
print("- Recovers sharp details lost in low-res version")
print("- Maintains consistency with downsampled observation")
print("- Produces plausible high-frequency content")
print("- Generates diverse solutions (hallucinated details)")

### Key Takeaways: Inverse Problems

This section demonstrated the **power of modular sampling for inverse problem research**:

**What We Showed:**
1. **Inpainting** - Recovered 50% missing pixels with DPS + FLUX
2. **Super-resolution** - 4x upscaling with sharp detail recovery
3. **Multiple solutions** - Stochastic sampling produces diverse plausible completions

**Why This Matters:**

✅ **20 lines of code** - Define your forward model (measurement operator)  
✅ **Plug into DPS** - Works with any integrator (Euler, DDIM, Heun, DPM++2S)  
✅ **Test on FLUX** - Or Stable Diffusion, or your custom model  
✅ **No training** - Pure sampling research, no model modification

**The Research Paradigm:**

This is exactly the "algorithms on top of models" philosophy in action:
- ✅ **Medical imaging**: Define CT/MRI operator → immediate reconstruction
- ✅ **Deblurring**: Define blur kernel → deblur with FLUX  
- ✅ **Compressed sensing**: Define measurement matrix → recover from few samples
- ✅ **Novel guidance**: Implement custom constraints → guided generation

**From Idea to Results: Hours, Not Weeks**

Traditional approach:
1. Train model (weeks + $$$)
2. Modify training code for your problem
3. Retrain with new objective
4. Hope it works

Diffuse approach:
1. Implement `ForwardModel` (20 lines)
2. Create `DPSDenoiser` with your integrator
3. Call `generate()`
4. Done

**This is the future of diffusion research** - algorithmic innovation on pretrained models.

## Summary: The Power of Modular Sampling

This notebook demonstrated Diffuse's core philosophy: **separation of concerns enables rapid experimentation**.

### What We Explored

1. **FLUX as Flow Matching** - velocity field prediction {cite}`flux2024` {cite}`Liu2022` {cite}`Lipman2022` (crash course Eq. :eq:`eq:flow_interpolation`)
2. **Timers** - discretization schedules (VpTimer, FluxTimer with Möbius shift)
3. **Integrators** - ODE solvers (DDIM, Euler, Heun, DPM++2S, Euler-Maruyama)
4. **Deterministic vs Stochastic** - controlled randomness for diversity
5. **Inverse Problems** - inpainting and super-resolution with DPS {cite}`Chung2022`
6. **Modularity** - swap components without model retraining

### The No-Training Research Paradigm

**Key Insight**: Most diffusion research is about **algorithms on top of models**, not training:

- 🎨 Image editing (InstructPix2Pix, DiffEdit, Imagic)
- 🖼️ Inpainting & outpainting (RePaint, Blended Diffusion) - **demonstrated in this notebook**
- 🔍 Inverse problems (DPS, RED-diff, medical imaging, super-resolution) - **demonstrated in this notebook**
- 🎯 Controllable generation (ControlNet-style, regional control, semantic guidance)
- 📐 Novel sampling methods (better integrators, adaptive schedules, churning)
- ⚡ Distillation & acceleration (consistency models, few-step sampling)
- 🧩 Compositional generation (multi-condition, spatial control)

**All of these can be researched using pre-trained models.** No training required.

### Modularity = Research Velocity

With Diffuse:
- Change one line → test a new integrator
- Swap Timer → try adaptive scheduling
- Replace Denoiser → add guidance method
- Implement ForwardModel (20 lines!) → solve inverse problem

**Test ideas in hours, not weeks.**

### Recommended Configurations

| Use Case | Timer | Integrator | Notes |
|----------|-------|------------|-------|
| Fast preview | FluxTimer | Euler/DDIM | Quick iteration |
| Balanced | FluxTimer | DDIM | Good default |
| Max quality | FluxTimer | DPM++2S/Heun | Best visual results |
| Diverse samples | FluxTimer | Euler-Maruyama | Stochastic variation |
| Inverse problems | FluxTimer | Heun + DPS | Measurement consistency |

**Performance vs Quality:**
- DDIM/Euler: Baseline speed, good quality
- Heun/DPM++2S: 2x cost, 10-20% quality improvement (often worth it!)
- Euler-Maruyama: Same speed as Euler, adds diversity

### For Algorithm Developers

This framework is built for research on sampling:
- ✅ Novel ODE/SDE solvers → implement Integrator protocol
- ✅ Adaptive schedules → implement Timer protocol
- ✅ Guidance methods → extend Denoiser
- ✅ Inverse problems → implement ForwardModel (as shown with inpainting/super-res!)
- ✅ Works with FLUX, SD, custom models → just load weights

### The Algorithmic Frontier

The future of diffusion research is in **algorithms built on top** of pre-trained models. With Diffuse, you can:

**Experiment rapidly** - test sampling ideas without retraining  
**Focus on research** - no loading complexity, simple pipeline  
**Build on SOTA** - use FLUX, Stable Diffusion, or your own models  
**Prototype fast** - ideas to results in hours, not weeks

**From this notebook:**
- Defined inpainting forward model: **20 lines**
- Defined super-res forward model: **20 lines**  
- Tested on FLUX: **immediately**
- Results: **publication-ready**

**No training tax. Just algorithms.**

This framework supports research on sampling algorithms, guidance methods, inverse problems, and compositional generation. All without touching model weights.

Want to add your own model? Check out the GitHub repo for integration guides.  
Want to implement your own inverse problem? Copy the ForwardModel examples from this notebook!

## Cleanup

In [ ]:
# Release model components
loader.release_transformer()
loader.release_vae()
print("✓ Model components released")

## References

```{bibliography}
:filter: docname in docnames
```